***GENERATED CODE FOR classification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Incident_ID', 'transformation_label': 'String Indexer'}], 'feature': 'Incident_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '20210217TNKNK', 'max': '20221029TXAVA', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Incident_ID'}, {'feature_label': 'Incident_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Incident_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date', 'transformation_label': 'String Indexer'}], 'feature': 'Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/11/2022', 'max': '9/9/2022', 'missing': '0', 'distinct': '305'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date'}, {'feature_label': 'Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'School', 'transformation_label': 'String Indexer'}], 'feature': 'School', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A. Philip Randolph Elementary School', 'max': 'Yeshiva Toras Chaim High School', 'missing': '0', 'distinct': '481'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'School'}, {'feature_label': 'School', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('School')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Victims_Killed', 'transformation_label': 'String Indexer'}], 'feature': 'Victims_Killed', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.19', 'stddev': '1.02', 'min': '0', 'max': '4', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Victims_Killed'}, {'feature_label': 'Victims_Killed', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Victims_Killed')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Victims_Wounded', 'transformation_label': 'String Indexer'}], 'feature': 'Victims_Wounded', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.67', 'stddev': '1.34', 'min': '0', 'max': '7', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Victims_Wounded'}, {'feature_label': 'Victims_Wounded', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Victims_Wounded')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Number_Victims', 'transformation_label': 'String Indexer'}], 'feature': 'Number_Victims', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.86', 'stddev': '2.09', 'min': '0', 'max': '9', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Number_Victims'}, {'feature_label': 'Number_Victims', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Number_Victims')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Shooter_Killed', 'transformation_label': 'String Indexer'}], 'feature': 'Shooter_Killed', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.04', 'stddev': '0.19', 'min': '0', 'max': '1', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Shooter_Killed'}, {'feature_label': 'Shooter_Killed', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Shooter_Killed')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Media_Attention', 'transformation_label': 'String Indexer'}], 'feature': 'Media_Attention', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '497', 'mean': '', 'stddev': '', 'min': 'International', 'max': 'Regional', 'missing': '3', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Media_Attention'}, {'feature_label': 'Media_Attention', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Media_Attention')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Quarter', 'transformation_label': 'String Indexer'}], 'feature': 'Quarter', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '492', 'mean': '', 'stddev': '', 'min': 'Fall', 'max': 'Winter', 'missing': '8', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Quarter'}, {'feature_label': 'Quarter', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Quarter')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Abbeville', 'max': 'Yakima', 'missing': '0', 'distinct': '295'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WV', 'missing': '0', 'distinct': '47'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'School_Level', 'transformation_label': 'String Indexer'}], 'feature': 'School_Level', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '492', 'mean': '', 'stddev': '', 'min': '6-12', 'max': 'Other', 'missing': '8', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'School_Level'}, {'feature_label': 'School_Level', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('School_Level')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location', 'transformation_label': 'String Indexer'}], 'feature': 'Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Basketball Court', 'max': 'School Bus', 'missing': '0', 'distinct': '20'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location'}, {'feature_label': 'Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Location_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Both Inside/Outside', 'max': 'School Bus', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location_Type'}, {'feature_label': 'Location_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'During_School', 'transformation_label': 'String Indexer'}], 'feature': 'During_School', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '496', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '4', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'During_School'}, {'feature_label': 'During_School', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('During_School')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Time_Period', 'transformation_label': 'String Indexer'}], 'feature': 'Time_Period', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '494', 'mean': '', 'stddev': '', 'min': 'After School', 'max': 'Sport Event', 'missing': '6', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Time_Period'}, {'feature_label': 'Time_Period', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Time_Period')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'First_Shot', 'transformation_label': 'String Indexer'}], 'feature': 'First_Shot', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '478', 'mean': '', 'stddev': '', 'min': '0:05:00', 'max': '9:59:00', 'missing': '22', 'distinct': '201'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'First_Shot'}, {'feature_label': 'First_Shot', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('First_Shot')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Duration_min', 'transformation_label': 'String Indexer'}], 'feature': 'Duration_min', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '489', 'mean': '3.69', 'stddev': '32.58', 'min': '1', 'max': '90', 'missing': '11', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Duration_min'}, {'feature_label': 'Duration_min', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Duration_min')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Summary', 'transformation_label': 'String Indexer'}], 'feature': 'Summary', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '"Student fatally shot another student during dispute over stolen ""ghost gun"""',
                                                                                                                                                                                                                                                                'max': 'Woman was upset school bus was late and pulled gun during argument with bus driver', 'missing': '0', 'distinct': '494'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Summary'}, {'feature_label': 'Summary', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Summary')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Situation', 'transformation_label': 'String Indexer'}], 'feature': 'Situation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '343', 'mean': '', 'stddev': '', 'min': 'Accidental', 'max': 'Suicide/Attempted', 'missing': '157', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Situation'}, {'feature_label': 'Situation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Situation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Targets', 'transformation_label': 'String Indexer'}], 'feature': 'Targets', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '410', 'mean': '', 'stddev': '', 'min': 'Both', 'max': 'Victims Targeted', 'missing': '90', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Targets'}, {'feature_label': 'Targets', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Targets')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Accomplice', 'transformation_label': 'String Indexer'}], 'feature': 'Accomplice', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '435', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '65', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Accomplice'}, {'feature_label': 'Accomplice', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Accomplice')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Hostages', 'transformation_label': 'String Indexer'}], 'feature': 'Hostages', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '487', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '13', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Hostages'}, {'feature_label': 'Hostages', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Hostages')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Barricade', 'transformation_label': 'String Indexer'}], 'feature': 'Barricade', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '493', 'mean': '', 'stddev': '', 'min': 'N', 'max': 'Yes', 'missing': '7', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Barricade'}, {'feature_label': 'Barricade', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Barricade')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Bullied', 'transformation_label': 'String Indexer'}], 'feature': 'Bullied', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '446', 'mean': '', 'stddev': '', 'min': 'N', 'max': 'Yes', 'missing': '54', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Bullied'}, {'feature_label': 'Bullied', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Bullied')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Domestic_Violence', 'transformation_label': 'String Indexer'}], 'feature': 'Domestic_Violence', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '470', 'mean': '', 'stddev': '', 'min': 'NO', 'max': 'Yes', 'missing': '30', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Domestic_Violence'}, {'feature_label': 'Domestic_Violence', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Domestic_Violence')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Active_Shooter_FBI', 'transformation_label': 'String Indexer'}], 'feature': 'Active_Shooter_FBI', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '496', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '4', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Active_Shooter_FBI'}, {'feature_label': 'Active_Shooter_FBI', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Active_Shooter_FBI')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run classificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	shooting = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Shooting.csv', 'filename': 'Shooting.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(shooting)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(shooting,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Incident_ID", "transformation_label": "String Indexer"}], "feature": "Incident_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "20210217TNKNK", "max": "20221029TXAVA", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Incident_ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.86", "stddev": "3.12", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "15.75", "stddev": "8.57", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "Day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2021.53", "stddev": "0.5", "min": "2021", "max": "2022", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"feature_label": "Date", "transformation_label": "String Indexer"}], "feature": "Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/11/2022", "max": "9/9/2022", "missing": "0", "distinct": "305"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date"}, {"transformationsData": [{"feature_label": "School", "transformation_label": "String Indexer"}], "feature": "School", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A. Philip Randolph Elementary School", "max": "Yeshiva Toras Chaim High School", "missing": "0", "distinct": "481"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "School"}, {"transformationsData": [{"feature_label": "Victims_Killed", "transformation_label": "String Indexer"}], "feature": "Victims_Killed", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.19", "stddev": "1.02", "min": "0", "max": "4", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Victims_Killed"}, {"transformationsData": [{"feature_label": "Victims_Wounded", "transformation_label": "String Indexer"}], "feature": "Victims_Wounded", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.67", "stddev": "1.34", "min": "0", "max": "7", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Victims_Wounded"}, {"transformationsData": [{"feature_label": "Number_Victims", "transformation_label": "String Indexer"}], "feature": "Number_Victims", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.86", "stddev": "2.09", "min": "0", "max": "9", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Number_Victims"}, {"transformationsData": [{"feature_label": "Shooter_Killed", "transformation_label": "String Indexer"}], "feature": "Shooter_Killed", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.04", "stddev": "0.19", "min": "0", "max": "1", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Shooter_Killed"}, {"transformationsData": [{"feature_label": "Media_Attention", "transformation_label": "String Indexer"}], "feature": "Media_Attention", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "497", "mean": "", "stddev": "", "min": "International", "max": "Regional", "missing": "3", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Media_Attention"}, {"transformationsData": [{"feature_label": "Quarter", "transformation_label": "String Indexer"}], "feature": "Quarter", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "492", "mean": "", "stddev": "", "min": "Fall", "max": "Winter", "missing": "8", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Quarter"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Abbeville", "max": "Yakima", "missing": "0", "distinct": "295"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "WV", "missing": "0", "distinct": "47"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "School_Level", "transformation_label": "String Indexer"}], "feature": "School_Level", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "492", "mean": "", "stddev": "", "min": "6-12", "max": "Other", "missing": "8", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "School_Level"}, {"transformationsData": [{"feature_label": "Location", "transformation_label": "String Indexer"}], "feature": "Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Basketball Court", "max": "School Bus", "missing": "0", "distinct": "20"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location"}, {"transformationsData": [{"feature_label": "Location_Type", "transformation_label": "String Indexer"}], "feature": "Location_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Both Inside/Outside", "max": "School Bus", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location_Type"}, {"transformationsData": [{"feature_label": "During_School", "transformation_label": "String Indexer"}], "feature": "During_School", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "496", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "4", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "During_School"}, {"transformationsData": [{"feature_label": "Time_Period", "transformation_label": "String Indexer"}], "feature": "Time_Period", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "494", "mean": "", "stddev": "", "min": "After School", "max": "Sport Event", "missing": "6", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Time_Period"}, {"transformationsData": [{"feature_label": "First_Shot", "transformation_label": "String Indexer"}], "feature": "First_Shot", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "478", "mean": "", "stddev": "", "min": "0:05:00", "max": "9:59:00", "missing": "22", "distinct": "201"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "First_Shot"}, {"transformationsData": [{"feature_label": "Duration_min", "transformation_label": "String Indexer"}], "feature": "Duration_min", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "489", "mean": "3.69", "stddev": "32.58", "min": "1", "max": "90", "missing": "11", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Duration_min"}, {"transformationsData": [{"feature_label": "Summary", "transformation_label": "String Indexer"}], "feature": "Summary", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "\"Student fatally shot another student during dispute over stolen \"\"ghost gun\"\"\"", "max": "Woman was upset school bus was late and pulled gun during argument with bus driver", "missing": "0", "distinct": "494"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Summary"}, {"transformationsData": [{"feature_label": "Situation", "transformation_label": "String Indexer"}], "feature": "Situation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "343", "mean": "", "stddev": "", "min": "Accidental", "max": "Suicide/Attempted", "missing": "157", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Situation"}, {"transformationsData": [{"feature_label": "Targets", "transformation_label": "String Indexer"}], "feature": "Targets", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "410", "mean": "", "stddev": "", "min": "Both", "max": "Victims Targeted", "missing": "90", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Targets"}, {"transformationsData": [{"feature_label": "Accomplice", "transformation_label": "String Indexer"}], "feature": "Accomplice", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "435", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "65", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Accomplice"}, {"transformationsData": [{"feature_label": "Hostages", "transformation_label": "String Indexer"}], "feature": "Hostages", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "487", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "13", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Hostages"}, {"transformationsData": [{"feature_label": "Barricade", "transformation_label": "String Indexer"}], "feature": "Barricade", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "493", "mean": "", "stddev": "", "min": "N", "max": "Yes", "missing": "7", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Barricade"}, {"transformationsData": [{"feature_label": "Bullied", "transformation_label": "String Indexer"}], "feature": "Bullied", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "446", "mean": "", "stddev": "", "min": "N", "max": "Yes", "missing": "54", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Bullied"}, {"transformationsData": [{"feature_label": "Domestic_Violence", "transformation_label": "String Indexer"}], "feature": "Domestic_Violence", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "470", "mean": "", "stddev": "", "min": "NO", "max": "Yes", "missing": "30", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Domestic_Violence"}, {"transformationsData": [{"feature_label": "Active_Shooter_FBI", "transformation_label": "String Indexer"}], "feature": "Active_Shooter_FBI", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "496", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "4", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Active_Shooter_FBI"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["Month", "Day", "Year", "Incident_ID_stringindexer", "Date_stringindexer", "School_stringindexer", "Victims_Killed_stringindexer", "Victims_Wounded_stringindexer", "Number_Victims_stringindexer", "Shooter_Killed_stringindexer", "Media_Attention_stringindexer", "Quarter_stringindexer", "City_stringindexer", "State_stringindexer", "School_Level_stringindexer", "Location_Type_stringindexer", "During_School_stringindexer", "Time_Period_stringindexer", "First_Shot_stringindexer", "Duration_min_stringindexer", "Summary_stringindexer", "Situation_stringindexer", "Targets_stringindexer", "Accomplice_stringindexer", "Hostages_stringindexer", "Barricade_stringindexer", "Bullied_stringindexer", "Domestic_Violence_stringindexer", "Active_Shooter_FBI_stringindexer"], "Location_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

